# Flow-Xl with Python 
### Need exported frame from the Frame Selector and Powder Extraction Tools from CrysAlisPro

## Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib as mpl
import pandas as pd        
import math
import tifffile
import re

import sys
!{sys.executable} -m pip install pyFAI
import pyFAI
from pyFAI.gui import jupyter

!{sys.executable} -m pip install pybaselines
from pybaselines import Baseline
from pybaselines.utils import gaussian

pi = math.pi

# Load calibration file
### calibrated using pyFAI-calib2 tool

### pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
#load mask for detector frame
oneD_mask = np.load("C:/mask/file/location/mask.npy")

In [ ]:
# load the calibration .poni file with incorrect detector distance
calibration = pyFAI.load('C:/poni/file/location/calibration.poni')
calibration

In [ ]:
# specify the correct detector distance (in mm)

new_dist1 = 60.136392395540156


In [ ]:
# set the correct distance

calibration.set_dist(new_dist1 / 1000)
calibration

## importing detector frames

In [ ]:
collection_dir = "D:/path/to/your/data/directory/"

processing_dir = "D:/path/to/your/processing/directory/"

collection_name = "your_dataset_name.tiff"

collection = os.path.join(collection_dir,collection_name)

original_pattern = np.loadtxt(os.path.join(processing_dir,"GLY9_1_10Threshold_11_incorrectDist.xy"))
original_pattern

In [ ]:
# read tiff file

tiff = tifffile.imread(collection)
tiff

In [ ]:
# display overlaid frame

plt.figure(figsize=(14, 14))

plt.imshow(tiff, cmap='gray', vmin=0, vmax=np.max(tiff))
plt.colorbar()
plt.title(collection_name[:-5])  



# Integration using pyFAI module
### pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
# integration with the corrected detector distance
pattern1 = calibration.integrate1d(tiff, 1500, unit=pyFAI.units.TTH_DEG, radial_range=[2,43], mask=oneD_mask) 

x1 = pattern1[0]
y1 = pattern1[1]

baseline_fitter1 = Baseline(x_data=x1)
pattern1

# Baseline correction
### using https://pybaselines.readthedocs.io/en/latest/ mor() baseline correction

In [ ]:
# display baseline correction fitting against raw integrated pattern
# modify half_window to change best fit

half_window = 35


# Create a new figure
plt.figure(figsize=(14, 14))
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.ylabel('Intensity')
plt.xlim(4, 40)

plt.plot(x1, y1, label=data_name1)
plt.plot(x1, baseline_fitter1.mor(y1, half_window=half_window)[0], label=format(f'{data_name1} w/ half window= {half_window}'))


corrected_data1 = y1 - baseline_fitter1.mor(y1, half_window=half_window)[0]

plt.plot(x1, corrected_data1, label=format(f'Corrected data {data_name1} w/ half window= {half_window}'))

plt.legend()


In [ ]:
dataset_path1= processing_dir + data_name1 +".xy"
print(dataset_path1)

In [ ]:
# export processed data as an .xy file
np.savetxt(dataset_path1, np.c_[x1,corrected_data1])